In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [2]:
aug = pd.read_csv('data/DSL_2021_08.csv')
distances = pd.read_csv('data/Distances.csv', sep = ';')
repairs = pd.read_csv('data/Repairs_2021_08-09.csv')

In [3]:
distances=distances.rename(columns={"StCodeFrom": "operSt", "StCodeTo": "operDestSt"})
distances['operDestSt']=pd.to_numeric(distances['operDestSt'], errors='coerce')
distances['operSt']=pd.to_numeric(distances['operSt'], errors='coerce')
distances=distances.dropna(axis=0).reset_index(drop=True).astype('int64')
# в distances имеются строчки со словом "нет" вместо кода станции - поэтому исправление ошибок
# так же тип object есть, что при мерджинге мешает соединить таблицу с aug

# REPAIRS

In [4]:
new = repairs[['carNum','repType','repBeginDate','repStCode']]
new = new[new.repType =='Текущий']
new['repBeginDate']=pd.to_datetime(new['repBeginDate'], format='%Y-%m-%dT').astype(str)
new

,carNum,repType,repBeginDate,repStCode
1,28068658,Текущий,2021-08-01 00:30:00,84000
4,52727617,Текущий,2021-08-02 22:11:00,83020
7,61819488,Текущий,2021-09-19 18:38:00,29050
9,61819611,Текущий,2021-08-01 20:53:00,83020
10,61819645,Текущий,2021-09-29 16:18:00,6150
...,...,...,...,...
3409,61819249,Текущий,2021-09-16 21:57:00,83020
3410,61819280,Текущий,2021-08-08 22:00:00,43830
3413,56897630,Текущий,2021-09-05 14:52:00,1300
3418,52727575,Текущий,2021-08-17 06:06:00,83020


Выбираю важные по моему мнению:
'ПРИБ ','ОТПР ','БРОС ','ИСКП ','ВКЛП ','РАСФ ','РПНП ','РМНТ ','ФОРМ ','ПВТП ','ПРСЛ '

In [5]:
vag = aug[aug['operCode'].isin(['ПРИБ ','ОТПР ','БРОС ','ИСКП ','ВКЛП ','РАСФ ','РПНП ','РМНТ ','ФОРМ ','ПВТП ','ПРСЛ '])]
vag['operDate']=pd.to_datetime(vag['operDate'], format='%Y-%m-%dT').astype(str)

In [6]:
list_of_carNum_in_rep=new['carNum'].unique()
len(list_of_carNum_in_rep)

2318

In [7]:
df2 = pd.DataFrame()
for i in range(len(list_of_carNum_in_rep)):
    x=list_of_carNum_in_rep[i]
    lst_dates_new =  new[new['carNum'] == x]['repBeginDate'].values
    lst_stations_new = new[new['carNum'] == x]['repStCode'].values
    print(i)
    

    new_lst = np.dstack((lst_dates_new,lst_stations_new))[0]
    
    lst_dates_vag = vag[vag['carNum'] == x]['operDate'].values
    lst_stations_vag = vag[vag['carNum'] == x]['operSt'].values

    vag_lst = np.dstack((lst_dates_vag,lst_stations_vag))[0]
    
    new_lst_set = set([tuple(x) for x in new_lst])
    vag_lst_set = set([tuple(x) for x in vag_lst])
    intersection = np.array([x for x in new_lst_set & vag_lst_set])
    print(intersection)
    if len(intersection)>0:
        mask = vag['carNum'].isin([x]) & vag['operDate'].isin(intersection[:,0]) & vag['operSt'].isin(intersection[:,1].astype(int))
        df2 = pd.concat([df2, vag.where(mask).dropna()], ignore_index=True)

0
[['2021-08-01 00:30:00' '84000']]
1
[['2021-08-02 22:11:00' '83020']]
2
[]
3
[['2021-08-01 20:53:00' '83020']]
4
[]
5
[]
6
[['2021-08-07 14:55:00' '31000']]
7
[]
8
[]
9
[['2021-08-04 18:32:00' '87000']]
10
[]
11
[]
12
[['2021-08-05 10:34:00' '43850']]
13
[]
14
[]
15
[]
16
[]
17
[['2021-08-30 11:20:00' '3300']]
18
[['2021-08-12 18:41:00' '18290']]
19
[['2021-08-02 12:20:00' '87110']]
20
[['2021-08-27 16:38:00' '4640']]
21
[['2021-08-27 18:51:00' '17430']]
22
[]
23
[['2021-08-02 01:19:00' '5000']]
24
[['2021-08-05 09:20:00' '21600']]
25
[['2021-08-21 04:33:00' '17430']]
26
[['2021-08-29 22:43:00' '4640']
 ['2021-08-29 07:25:00' '4640']]
27
[['2021-08-23 19:14:00' '4000']]
28
[['2021-08-26 19:04:00' '4640']]
29
[['2021-08-28 21:01:00' '61700']]
30
[]
31
[]
32
[['2021-08-05 01:16:00' '30750']]
33
[['2021-08-16 13:17:00' '77850']]
34
[]
35
[]
36
[]
37
[]
38
[]
39
[['2021-08-23 20:05:00' '80660']]
40
[]
41
[]
42
[]
43
[['2021-08-17 15:12:00' '19000']]
44
[['2021-08-29 12:24:00' '83020']]
4

369
[['2021-08-31 18:19:00' '18450']]
370
[]
371
[['2021-08-22 10:01:00' '98830']]
372
[['2021-08-19 18:03:00' '18290']]
373
[]
374
[]
375
[]
376
[['2021-08-18 13:43:00' '94910']]
377
[['2021-08-12 16:44:00' '19000']]
378
[]
379
[['2021-08-09 12:09:00' '23860']]
380
[]
381
[['2021-08-17 15:13:00' '19000']]
382
[['2021-08-17 15:12:00' '19000']]
383
[]
384
[]
385
[['2021-08-14 16:08:00' '19000']]
386
[['2021-08-13 03:57:00' '98080']]
387
[['2021-08-24 18:22:00' '98590']]
388
[]
389
[['2021-08-19 19:15:00' '10080']]
390
[['2021-08-23 05:06:00' '98080']]
391
[['2021-08-02 07:28:00' '89210']]
392
[]
393
[['2021-08-22 11:30:00' '85160']]
394
[]
395
[]
396
[['2021-08-02 17:11:00' '18290']]
397
[['2021-08-16 11:58:00' '89210']]
398
[]
399
[['2021-08-16 12:03:00' '89210']]
400
[]
401
[['2021-08-03 12:04:00' '59300']]
402
[]
403
[['2021-08-18 20:24:00' '22560']]
404
[['2021-08-14 01:06:00' '5000']]
405
[['2021-08-02 03:58:00' '20000']]
406
[['2021-08-11 12:08:00' '3000']
 ['2021-08-03 21:31:00' 

730
[]
731
[]
732
[]
733
[]
734
[]
735
[['2021-08-29 23:01:00' '30750']]
736
[['2021-08-18 18:07:00' '30110']]
737
[['2021-08-23 18:32:00' '87190']]
738
[['2021-08-10 08:54:00' '83020']]
739
[['2021-08-26 08:20:00' '93690']]
740
[['2021-08-04 04:07:00' '5000']]
741
[['2021-08-23 18:56:00' '3000']]
742
[['2021-08-03 10:41:00' '51310']]
743
[['2021-08-21 06:19:00' '18000']]
744
[['2021-08-25 18:25:00' '21000']]
745
[]
746
[]
747
[]
748
[]
749
[]
750
[['2021-08-30 21:16:00' '3000']]
751
[['2021-08-11 03:52:00' '4000']]
752
[]
753
[]
754
[]
755
[]
756
[]
757
[]
758
[]
759
[['2021-08-20 23:47:00' '83020']]
760
[['2021-08-08 12:59:00' '59300']]
761
[['2021-08-02 22:11:00' '83020']]
762
[]
763
[['2021-08-10 16:35:00' '19000']]
764
[['2021-08-13 10:01:00' '88380']]
765
[]
766
[]
767
[['2021-08-15 09:28:00' '86380']]
768
[['2021-08-13 01:08:00' '96820']]
769
[['2021-08-17 14:09:00' '58510']]
770
[['2021-08-28 22:19:00' '3000']
 ['2021-08-08 18:44:00' '29000']]
771
[]
772
[]
773
[]
774
[['2021-0

1084
[]
1085
[['2021-08-03 17:56:00' '93000']]
1086
[['2021-08-07 20:07:00' '51310']]
1087
[]
1088
[['2021-08-03 09:53:00' '51230']]
1089
[]
1090
[]
1091
[['2021-08-07 05:51:00' '30000']]
1092
[['2021-08-24 18:16:00' '28000']]
1093
[]
1094
[]
1095
[]
1096
[['2021-08-17 19:06:00' '28000']]
1097
[['2021-08-16 13:17:00' '77850']]
1098
[]
1099
[['2021-08-01 18:19:00' '87330']]
1100
[]
1101
[]
1102
[]
1103
[]
1104
[['2021-08-27 17:25:00' '78000']]
1105
[]
1106
[['2021-08-15 09:30:00' '86380']]
1107
[['2021-08-11 11:23:00' '59640']]
1108
[]
1109
[]
1110
[]
1111
[]
1112
[['2021-08-09 18:28:00' '53550']]
1113
[]
1114
[]
1115
[]
1116
[]
1117
[]
1118
[['2021-08-06 09:07:00' '59300']]
1119
[]
1120
[]
1121
[['2021-08-05 06:51:00' '83020']]
1122
[['2021-08-06 18:15:00' '21000']]
1123
[['2021-08-26 23:57:00' '17430']]
1124
[]
1125
[['2021-08-05 10:07:00' '51310']]
1126
[['2021-08-24 19:28:00' '17430']]
1127
[]
1128
[['2021-08-14 18:15:00' '23860']]
1129
[]
1130
[]
1131
[]
1132
[]
1133
[]
1134
[]
113

1436
[['2021-08-24 16:08:00' '58200']]
1437
[['2021-08-03 12:14:00' '83020']]
1438
[]
1439
[]
1440
[]
1441
[['2021-08-18 18:08:00' '28000']]
1442
[['2021-08-15 20:46:00' '28000']]
1443
[]
1444
[]
1445
[]
1446
[]
1447
[['2021-08-16 19:14:00' '28000']]
1448
[]
1449
[]
1450
[]
1451
[]
1452
[]
1453
[['2021-08-13 02:12:00' '77850']]
1454
[['2021-08-03 06:51:00' '86220']]
1455
[]
1456
[]
1457
[]
1458
[['2021-08-15 10:26:00' '4000']]
1459
[]
1460
[['2021-08-31 17:57:00' '58000']]
1461
[]
1462
[]
1463
[]
1464
[]
1465
[]
1466
[]
1467
[['2021-08-15 09:30:00' '86380']]
1468
[['2021-08-06 18:08:00' '18290']]
1469
[['2021-08-17 07:49:00' '89210']]
1470
[]
1471
[]
1472
[['2021-08-22 05:51:00' '21000']]
1473
[['2021-08-02 06:34:00' '83020']]
1474
[['2021-08-20 23:38:00' '52160']]
1475
[]
1476
[]
1477
[]
1478
[]
1479
[['2021-08-08 18:28:00' '21000']]
1480
[['2021-08-10 21:42:00' '77850']]
1481
[['2021-08-03 10:37:00' '92310']]
1482
[]
1483
[['2021-08-03 10:36:00' '92310']]
1484
[['2021-08-06 01:37:00'

1811
[]
1812
[]
1813
[]
1814
[]
1815
[['2021-08-09 18:27:00' '53550']]
1816
[]
1817
[]
1818
[]
1819
[['2021-08-12 18:07:00' '25490']]
1820
[]
1821
[['2021-08-06 05:27:00' '17270']]
1822
[]
1823
[]
1824
[]
1825
[['2021-08-08 22:14:00' '80660']]
1826
[['2021-08-14 11:21:00' '52440']]
1827
[['2021-08-10 17:56:00' '19260']]
1828
[['2021-08-28 21:43:00' '43850']]
1829
[]
1830
[]
1831
[['2021-08-25 18:13:00' '28000']]
1832
[['2021-08-07 19:19:00' '3000']]
1833
[['2021-08-25 17:50:00' '19260']]
1834
[]
1835
[]
1836
[['2021-08-05 09:47:00' '23000']]
1837
[['2021-08-30 01:40:00' '17270']]
1838
[]
1839
[]
1840
[]
1841
[]
1842
[['2021-08-17 14:56:00' '23000']]
1843
[['2021-08-17 20:55:00' '43850']]
1844
[]
1845
[]
1846
[]
1847
[['2021-08-10 18:59:00' '5000']]
1848
[]
1849
[['2021-08-08 04:22:00' '29050']]
1850
[['2021-08-02 23:49:00' '98450']]
1851
[['2021-08-09 09:58:00' '83020']]
1852
[['2021-08-24 02:49:00' '30460']]
1853
[]
1854
[]
1855
[['2021-08-15 20:48:00' '28000']]
1856
[['2021-08-12 20:

2165
[]
2166
[]
2167
[]
2168
[]
2169
[]
2170
[['2021-08-27 18:17:00' '30000']]
2171
[['2021-08-21 17:32:00' '28000']]
2172
[['2021-08-23 03:51:00' '96000']]
2173
[]
2174
[]
2175
[['2021-08-30 06:52:00' '4000']]
2176
[['2021-08-26 13:57:00' '17640']]
2177
[['2021-08-29 18:15:00' '24420']]
2178
[]
2179
[['2021-08-21 19:18:00' '26000']]
2180
[['2021-08-30 06:40:00' '30000']
 ['2021-08-04 22:29:00' '84000']]
2181
[]
2182
[]
2183
[['2021-08-23 23:43:00' '77470']]
2184
[['2021-08-20 21:17:00' '3000']]
2185
[]
2186
[['2021-08-16 09:12:00' '62560']]
2187
[]
2188
[['2021-08-19 17:59:00' '26000']]
2189
[['2021-08-09 19:12:00' '85000']]
2190
[['2021-08-19 13:25:00' '81620']]
2191
[]
2192
[['2021-08-30 08:26:00' '58720']]
2193
[]
2194
[['2021-08-19 15:14:00' '31000']]
2195
[['2021-08-04 08:38:00' '28000']]
2196
[]
2197
[]
2198
[['2021-08-20 17:44:00' '23550']]
2199
[['2021-08-31 10:03:00' '28090']]
2200
[]
2201
[]
2202
[['2021-08-13 09:35:00' '4000']]
2203
[]
2204
[]
2205
[['2021-08-13 06:38:00' '

In [8]:
df2

,carNum,operDate,operCode,operSt,operDestSt,operCargo,operWeight
0,28068658.0,2021-08-01 00:30:00,РМНТ,84000.0,84540.0,66209,0.0
1,52727617.0,2021-08-02 22:11:00,РМНТ,83020.0,86230.0,30101,0.0
2,61819611.0,2021-08-01 20:53:00,РМНТ,83020.0,86230.0,23225,0.0
3,59734285.0,2021-08-07 14:55:00,РМНТ,31000.0,31110.0,08108,0.0
4,61819710.0,2021-08-04 18:32:00,РМНТ,87000.0,87610.0,16120,0.0
...,...,...,...,...,...,...,...
1153,61819165.0,2021-08-13 10:03:00,РМНТ,28570.0,28570.0,23225,0.0
1154,61819413.0,2021-08-26 23:56:00,РМНТ,17430.0,17270.0,23245,0.0
1155,61819280.0,2021-08-08 22:00:00,РМНТ,43830.0,46700.0,16101,69.0
1156,52727575.0,2021-08-29 18:18:00,РМНТ,86290.0,86290.0,23239,0.0


In [9]:
df2['operCode'].value_counts()

РМНТ     1158
Name: operCode, dtype: int64

In [10]:
#РМНТ - соответсвует датам текущего ремонта для вагона

In [13]:
df2.to_csv('current_repair.csv')